In [19]:
# Import necessary libraries
import pandas as pd
import requests

# Define the API endpoints for the data
current_quiz_data_url = "https://www.jsonkeeper.com/b/LLQT"  
historical_quiz_data_url = "https://api.jsonserve.com/XgAgFJ"  

# Step 1: Load the Current Quiz Data
response_current = requests.get(current_quiz_data_url)
if response_current.status_code == 200:
    current_quiz_data = response_current.json()  # Convert JSON response to Python dictionary
    current_df = pd.json_normalize(current_quiz_data)  # Convert to DataFrame
    print("Current Quiz Data Loaded Successfully")
else:
    print(f"Failed to load current quiz data. Status code: {response_current.status_code}")
    current_df = pd.DataFrame()  # Initialize as empty DataFrame

# Step 2: Load the Historical Quiz Data
response_historical = requests.get(historical_quiz_data_url)
if response_historical.status_code == 200:
    historical_quiz_data = response_historical.json()  # Convert JSON response to Python dictionary
    historical_df = pd.json_normalize(historical_quiz_data)  # Convert to DataFrame
    print("Historical Quiz Data Loaded Successfully")
else:
    print(f"Failed to load historical quiz data. Status code: {response_historical.status_code}")
    historical_df = pd.DataFrame()  # Initialize as empty DataFrame

# Step 3: Inspect the DataFrames
print("Current Quiz DataFrame:")
print(current_df.head())  # Preview the first few rows of Current Quiz Data

print("Historical Quiz DataFrame:")
print(historical_df.head())  # Preview the first few rows of Historical Quiz Data

# Step 4: Analyze the Data (Example: Find topics with the lowest accuracy)
# Example analysis for Current Quiz Data
if 'response_accuracy' in current_df.columns:
    accuracy_analysis = current_df.groupby('topic')['response_accuracy'].mean().sort_values()
    print("Topics with Lowest Accuracy:")
    print(accuracy_analysis)

# Example analysis for Historical Quiz Data
if 'score' in historical_df.columns:
    score_analysis = historical_df.groupby('user_id')['score'].mean().sort_values()
    print("Users with Lowest Average Scores:")
    print(score_analysis)

# Step 5: Generate Insights
# Highlight weak areas (topics with low accuracy) for a specific user
specific_user_id = "user_123"  
if 'user_id' in current_df.columns and 'topic' in current_df.columns and 'response_accuracy' in current_df.columns:
    user_data = current_df[current_df['user_id'] == specific_user_id]  
    if not user_data.empty:
        user_weak_areas = user_data.groupby('topic')['response_accuracy'].mean()
        print(f"Weak Areas for User {specific_user_id}:")
        print(user_weak_areas)
    else:
        print(f"No data found for User {specific_user_id}")
        user_weak_areas = pd.Series(dtype=float)  
else:
    print("Required columns ('user_id', 'topic', 'response_accuracy') not found in Current Quiz Data")
    user_weak_areas = pd.Series(dtype=float)

# Step 6: Recommendations
# Generate recommendations based on weak topics
if not user_weak_areas.empty:
    recommendations = user_weak_areas[user_weak_areas < 0.5].index.tolist()  # Topics with accuracy < 50%
    if recommendations:
        print(f"Suggested Topics for Improvement for User {specific_user_id}: {recommendations}")
    else:
        print(f"No significant weak areas found for User {specific_user_id}.")
else:
    print(f"No weak areas identified for User {specific_user_id}.")

# Step 7: Save DataFrames
# Save the processed data as CSV for further exploration
current_df.to_csv('current_quiz_data.csv', index=False)
historical_df.to_csv('historical_quiz_data.csv', index=False)


Current Quiz Data Loaded Successfully
Historical Quiz Data Loaded Successfully
Current Quiz DataFrame:
   quiz.id quiz.name                                         quiz.title  \
0       43      None  Structural Organisation in Animals and Plants (7)   

  quiz.description quiz.difficulty_level                          quiz.topic  \
0                                   None  Structural Organisation in Animals   

                       quiz.time  quiz.is_published  \
0  2024-07-03T00:00:00.000+05:30               True   

                 quiz.created_at                quiz.updated_at  ...  \
0  2024-07-03T11:00:08.958+05:30  2024-09-23T18:43:27.751+05:30  ...   

   quiz.exam_id quiz.show_unanswered quiz.ends_at quiz.lives  quiz.live_count  \
0          None                False   2025-01-18       None        Free Test   

   quiz.coin_count  quiz.questions_count   quiz.daily_date  \
0               -1                   128  January 17, 2025   

   quiz.max_mistake_count                